In [1]:
# -*- coding: utf-8 -*-
"""GNN Model.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1pSO-VEyn5Cywjw9sXKn2fjXdVbI3hAsH
"""

# -*- coding: utf-8 -*-
"""Untitled2.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/193StgLnr4doKklAxwBiQsVX3njEfb1oa
"""

DATA_PATH = r"C:\Users\jamak\OneDrive\Thesis\Merged_Data.csv"

In [2]:
!pip install vmdpy
!pip install optuna
import optuna
from vmdpy import VMD 

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('/kaggle/input/vmdpy')  # Adjust the path if needed

warnings.filterwarnings('ignore')


# Check for GPU and set device appropriately for Kaggle T4
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set optimized CUDA options for T4 GPU
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False  # Better performance but less reproducible
    # Set to float16 precision for faster computation on T4 GPU
    torch.set_float32_matmul_precision('high')

# ============================================================
# MODIFIED DATA LOADING AND PROCESSING FOR TARGETED SEASONAL ANALYSIS
# ============================================================

def load_data(file_path):
    """
    Load weather data, clean missing values, and filter to LA Downtown only.
    """
    try:
        # Determine file type and read
        if file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path, engine='openpyxl')
        else:
            df = pd.read_csv(file_path, encoding='ISO-8859-1')
    except Exception as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame()  # Return empty DataFrame

    # Convert timestamp
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    elif 'DATE' in df.columns:
        df['timestamp'] = pd.to_datetime(df['DATE'])
        df = df.rename(columns={'DATE': 'date_original'})

    # Ensure time-based ordering before interpolation
    df = df.sort_values(by='timestamp')

    # Add hour of day feature - sine/cosine encoding for cyclical pattern
    df['hour_sin'] = np.sin(2 * np.pi * df['timestamp'].dt.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['timestamp'].dt.hour / 24)

    # Add day of year feature - sine/cosine encoding for cyclical pattern
    df['day_sin'] = np.sin(2 * np.pi * df['timestamp'].dt.dayofyear / 365.25)
    df['day_cos'] = np.cos(2 * np.pi * df['timestamp'].dt.dayofyear / 365.25)

    # Define region mapping for LA Downtown only
    region_mapping = {
        'LA Downtown': 'urban'
    }

    # Filter to keep only LA Downtown
    if 'station_id' in df.columns:
        # Filter to LA Downtown only
        df = df[df['station_id'] == 'LA Downtown']
        
        # Add region information
        df['region'] = df['station_id'].map(region_mapping)

        # Convert region to numerical encoding
        region_to_num = {region: i for i, region in enumerate(df['region'].unique())}
        df['region_code'] = df['region'].map(region_to_num)

        # Add elevation for LA Downtown
        elevation_mapping = {
            'LA Downtown': 93         # meters
        }
        df['elevation'] = df['station_id'].map(elevation_mapping)
        # Normalize elevation (will be constant for single station)
        df['elevation_norm'] = 0.0  # Since we only have one station, just use 0 as normalized value

        print(f"Filtered data to LA Downtown only")

    # Interpolate missing values along the time dimension
    df.interpolate(method='linear', limit_direction='both', inplace=True)

    return df

# ============================================================
# VMD DECOMPOSITION
# ============================================================

import multiprocessing
import joblib
import os

def perform_vmd_decomposition(signal, alpha=2000, tau=0, K=10, DC=0, init=1, tol=1e-7):
    """
    Perform VMD decomposition on a signal.
    """
    # Run VMD
    u, u_hat, omega = VMD(signal, alpha, tau, K, DC, init, tol)
    
    # Calculate reconstruction residual
    recon = np.sum(u, axis=0)
    rres = np.sqrt(np.mean((signal - recon)**2)) / np.sqrt(np.mean(signal**2)) * 100
    
    return u, rres

def find_optimal_k(signal, k_range=range(6, 15)):
    """
    Find optimal K where rres < 3% with no sharp drop.
    """
    results = []
    
    for k in k_range:
        print(f"  Testing K={k}")
        _, rres = perform_vmd_decomposition(signal, K=k)
        results.append((k, rres))
        print(f"  K={k}, rres={rres:.2f}%")
        if rres < 3.0:
            return k, rres
            
    # If no K achieves rres < 3%, return the K with lowest rres
    return min(results, key=lambda x: x[1])

def decompose_station_data(data, feature_name, station_id, cache_dir='vmd_cache'):
    """
    Decompose time series data for a single station.
    """
    print(f"Starting decomposition for {station_id} - {feature_name}")
    
    # Create cache directory if it doesn't exist
    os.makedirs(cache_dir, exist_ok=True)
    
    # Create a unique cache filename
    cache_file = f"{cache_dir}/vmd_{station_id}_{feature_name}.joblib"
    
    # Check if cached results exist
    if os.path.exists(cache_file):
        print(f"Loading cached VMD decomposition for {station_id} - {feature_name}")
        return joblib.load(cache_file)
    
    # Get the time series to decompose
    print(f"  Extracting signal for {station_id}")
    signal = data[feature_name].values
    
    # Normalize signal
    print(f"  Normalizing signal")
    signal_norm = (signal - np.mean(signal)) / np.std(signal)
    
    # Find optimal K
    print(f"  Finding optimal K")
    k_opt, rres = find_optimal_k(signal_norm)
    print(f"Station {station_id}, feature {feature_name}: Optimal K={k_opt}, rres={rres:.2f}%")
    
    # Perform VMD with optimal K
    print(f"  Performing VMD with K={k_opt}")
    u, _ = perform_vmd_decomposition(signal_norm, K=k_opt)
    
    # Denormalize modes
    print(f"  Denormalizing modes")
    u_denorm = u * np.std(signal) + np.mean(signal) / k_opt
    
    # Save to cache
    print(f"  Saving to cache: {cache_file}")
    joblib.dump(u_denorm, cache_file)
    
    print(f"Completed decomposition for {station_id} - {feature_name}")
    return u_denorm

def parallel_vmd_decomposition(df, feature_cols_to_decompose=['Temperature_C']):
    """
    Apply VMD decomposition to multiple stations serially to avoid multiprocessing issues.
    """
    # Get the unique station IDs 
    station_ids = ['LA Downtown']  # We only have LA Downtown in this code
    
    decomposed_data = {}
    
    for feature in feature_cols_to_decompose:
        print(f"Performing VMD decomposition for feature: {feature}")
        
        # Process each station serially instead of in parallel
        results = []
        for station_id in station_ids:
            print(f"  Processing station: {station_id}")
            result = decompose_station_data(df, feature, station_id)
            results.append(result)
            
        # Store results
        for i, station_id in enumerate(station_ids):
            if station_id not in decomposed_data:
                decomposed_data[station_id] = {}
            decomposed_data[station_id][feature] = results[i]
    
    return decomposed_data

def extract_target_days(df):
    """
    Extract the 4 specific target days (one per season) for predictions
    """
    # Define target days
    target_days = [
        {'season': 'Spring', 'month': 4, 'day': 15},  # April 15
        {'season': 'Summer', 'month': 7, 'day': 20},  # July 20
        {'season': 'Fall', 'month': 10, 'day': 10},   # October 10
        {'season': 'Winter', 'month': 1, 'day': 15}   # January 15
    ]

    # Filter for each target day
    target_data = {}
    for target in target_days:
        # Filter by month and day
        day_data = df[(df['timestamp'].dt.month == target['month']) &
                       (df['timestamp'].dt.day == target['day'])]

        # Get the most recent year that has data for this day
        if not day_data.empty:
            latest_year = day_data['timestamp'].dt.year.max()
            target_day_data = day_data[day_data['timestamp'].dt.year == latest_year]
            target_data[target['season']] = target_day_data
            print(f"Found {len(target_day_data)} records for {target['season']} target day ({target['month']}/{target['day']}/{latest_year})")
        else:
            print(f"WARNING: No data found for {target['season']} target day")

    return target_data

def prepare_seasonal_training_data(df, target_days):
    """
    For each target day, prepare all historical data for training
    """
    training_sets = {}

    for season, target_day_data in target_days.items():
        if target_day_data.empty:
            continue

        # Get the date of this target
        sample_date = target_day_data['timestamp'].iloc[0]
        target_year = sample_date.year

        # Use all historical data prior to the target year
        historical_data = df[df['timestamp'].dt.year < target_year]

        training_sets[season] = historical_data
        print(f"{season} training set: {len(historical_data)} samples from all historical data")

    return training_sets

def normalize_features(train_df, val_df, feature_cols):
    """
    Normalize features using StandardScaler fitted on training data
    """
    scaler = StandardScaler()

    # Fit on training data
    scaler.fit(train_df[feature_cols])

    # Transform datasets
    train_scaled = scaler.transform(train_df[feature_cols])
    val_scaled = scaler.transform(val_df[feature_cols])

    # Convert back to DataFrames
    train_norm = pd.DataFrame(train_scaled, columns=feature_cols, index=train_df.index)
    val_norm = pd.DataFrame(val_scaled, columns=feature_cols, index=val_df.index)

    return train_norm, val_norm, scaler

class WeatherDataset(Dataset):
    """
    Dataset for weather forecasting with sliding window approach.
    Modified to work efficiently with single station (LA Downtown).
    """
    def __init__(self, df, station_ids, feature_cols, seq_length=24, forecast_horizon=24):
        """
        Args:
            df: DataFrame with weather data (filtered to LA Downtown only)
            station_ids: List containing only 'LA Downtown'
            feature_cols: List of feature columns to use as input
            seq_length: Length of input sequence (in hours)
            forecast_horizon: How many hours ahead to predict
        """
        self.df = df
        self.station_ids = station_ids
        self.feature_cols = feature_cols
        self.seq_length = seq_length
        self.forecast_horizon = forecast_horizon
        self.n_stations = len(station_ids)  # Should be 1

        # Get unique timestamps (now all from one station)
        self.timestamps = sorted(df['timestamp'].unique())

        # Filter valid timestamps (those that have enough history and future data)
        valid_idx = []
        for i in range(len(self.timestamps) - (seq_length + forecast_horizon - 1)):
            # Check if we have continuous data for this window
            current_time = self.timestamps[i]
            end_time = self.timestamps[i + seq_length + forecast_horizon - 1]
            expected_duration = timedelta(hours=seq_length + forecast_horizon - 1)

            if (end_time - current_time) == expected_duration:
                valid_idx.append(i)

        self.valid_indices = valid_idx

        # Add a safety check to ensure we have at least one valid window
        if len(self.valid_indices) == 0:
            print(f"WARNING: No valid windows found in dataset. Using reduced requirements.")
            # Fall back to allowing any windows where we have both input and output data
            valid_idx = []
            for i in range(len(self.timestamps) - seq_length):
                if i + seq_length < len(self.timestamps):
                    valid_idx.append(i)
            self.valid_indices = valid_idx
            self.fallback_mode = True
            print(f"Found {len(self.valid_indices)} windows with relaxed continuity requirements")
        else:
            self.fallback_mode = False
            print(f"Created dataset with {len(self.valid_indices)} valid windows")

    def __len__(self):
        return max(1, len(self.valid_indices))  # Ensure length is at least 1

    def __getitem__(self, idx):
        if len(self.valid_indices) == 0:
            # Return dummy data if no valid indices
            X = np.zeros((len(self.feature_cols), self.n_stations, self.seq_length))
            y = np.zeros((self.n_stations, self.forecast_horizon))
            static_features = np.zeros((self.n_stations, 2))  # region_code and elevation
            return (torch.FloatTensor(X), torch.FloatTensor(static_features)), torch.FloatTensor(y)

        # Get actual data when possible
        start_idx = self.valid_indices[idx % len(self.valid_indices)]

        # Get timestamps for input and output windows
        input_timestamps = self.timestamps[start_idx:start_idx + self.seq_length]
        output_timestamps = self.timestamps[start_idx + self.seq_length:
                                            start_idx + self.seq_length + self.forecast_horizon]

        # Handle potential shortfall in output window (fallback mode)
        if self.fallback_mode and len(output_timestamps) < self.forecast_horizon:
            # Pad with repetition of last timestamp if needed
            last_time = output_timestamps[-1] if len(output_timestamps) > 0 else input_timestamps[-1]
            padding = [last_time] * (self.forecast_horizon - len(output_timestamps))
            output_timestamps = list(output_timestamps) + padding

        # Initialize tensors - simpler now with just one station
        X = np.zeros((len(self.feature_cols), 1, self.seq_length))  # Only one station
        y = np.zeros((1, self.forecast_horizon))  # Only one station
        static_features = np.zeros((1, 2))  # region_code and elevation

        # LA Downtown is our only station
        station_id = 'LA Downtown'
        
        # Input sequence
        for t_idx, ts in enumerate(input_timestamps):
            station_data = self.df[self.df['timestamp'] == ts]

            if not station_data.empty:
                for f_idx, feat in enumerate(self.feature_cols):
                    X[f_idx, 0, t_idx] = station_data[feat].values[0]

                # Store static features (same for all timestamps)
                if t_idx == 0:
                    static_features[0, 0] = station_data['region_code'].values[0]
                    static_features[0, 1] = station_data['elevation_norm'].values[0]

        # Target sequence (temperature only)
        for t_idx, ts in enumerate(output_timestamps):
            if t_idx < self.forecast_horizon:  # Safety check
                station_data = self.df[self.df['timestamp'] == ts]

                if not station_data.empty:
                    y[0, t_idx] = station_data['Temperature_C'].values[0]

        return (torch.FloatTensor(X), torch.FloatTensor(static_features)), torch.FloatTensor(y)
    

class TimeSeriesCVSplit:
    """
    Time series cross-validation with multiple train-validation folds.
    Supports both expanding window and sliding window approaches.
    """
    def __init__(self, timestamps, n_splits=5, test_size=0.15, gap=24, min_train_size=0.5):
        """
        Args:
            timestamps: Array of all timestamps in the dataset
            n_splits: Number of cross-validation folds
            test_size: Size of validation set in each fold (as fraction)
            gap: Minimum gap between train and validation sets (in hours)
            min_train_size: Minimum size of training set (as fraction)
        """
        self.timestamps = np.array(timestamps)
        self.n_splits = n_splits
        self.test_size = test_size
        self.gap = gap
        self.min_train_size = min_train_size
        
    def split(self):
        """Generate train/validation indices for each fold."""
        n_samples = len(self.timestamps)
        indices = np.arange(n_samples)
        
        # Size of each test fold
        test_size = int(n_samples * self.test_size)
        # Minimum training set size
        min_train_size = int(n_samples * self.min_train_size)
        
        # Generate folds
        for i in range(self.n_splits):
            # For the last fold, use the last test_size samples as validation
            if i == self.n_splits - 1:
                test_start = n_samples - test_size
            else:
                # Distribute validation sets evenly across the timeline
                test_start = min_train_size + i * ((n_samples - min_train_size - test_size) // max(1, self.n_splits - 1))
            
            test_end = test_start + test_size
            
            # Apply gap between train and test
            train_end = max(0, test_start - self.gap)
            
            # Get the actual indices
            train_indices = indices[:train_end]
            test_indices = indices[test_start:test_end]
            
            yield train_indices, test_indices
            
    def get_fold_timestamps(self):
        """Return the timestamp ranges for each fold (for visualization)."""
        fold_timestamps = []
        
        for train_idx, val_idx in self.split():
            train_start = self.timestamps[train_idx[0]]
            train_end = self.timestamps[train_idx[-1]]
            val_start = self.timestamps[val_idx[0]]
            val_end = self.timestamps[val_idx[-1]]
            
            fold_timestamps.append({
                'fold': len(fold_timestamps) + 1,
                'train_range': (train_start, train_end),
                'val_range': (val_start, val_end)
            })
            
        return fold_timestamps
# ============================================================
# TEMPORAL FUSION TRANSFORMER IMPLEMENTATION
# ============================================================
class TemporalSelfAttention(nn.Module):
    """
    Multi-head self-attention layer for temporal data.
    Simplified from the original TFT paper.
    """
    def __init__(self, d_model, n_heads=2, dropout=0.1):
        super(TemporalSelfAttention, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads

        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)

        self.out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, seq_length, _ = x.size()

        # Linear projections
        queries = self.query(x).view(batch_size, seq_length, self.n_heads, self.head_dim).transpose(1, 2)
        keys = self.key(x).view(batch_size, seq_length, self.n_heads, self.head_dim).transpose(1, 2)
        values = self.value(x).view(batch_size, seq_length, self.n_heads, self.head_dim).transpose(1, 2)

        # Attention scores
        scores = torch.matmul(queries, keys.transpose(-2, -1)) / np.sqrt(self.head_dim)
        attention = F.softmax(scores, dim=-1)
        attention = self.dropout(attention)

        # Apply attention to values
        out = torch.matmul(attention, values)
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

        # Final linear layer
        return self.out(out)

class GatedResidualNetwork(nn.Module):
    """
    Gated Residual Network as described in the TFT paper.
    Simplified version with fewer layers.
    """
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1):
        super(GatedResidualNetwork, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size

        # If input and output sizes are different, apply a skip connection
        self.skip_layer = None
        if input_size != output_size:
            self.skip_layer = nn.Linear(input_size, output_size)

        # Main layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.gate = nn.Linear(input_size + output_size, output_size)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(output_size)

    def forward(self, x):
        # Main branch
        hidden = F.elu(self.fc1(x))
        hidden = self.dropout(hidden)
        hidden = self.fc2(hidden)

        # Skip connection
        if self.skip_layer is not None:
            skip = self.skip_layer(x)
        else:
            skip = x

        # Gate mechanism
        gate_input = torch.cat([x, hidden], dim=-1)
        gate = torch.sigmoid(self.gate(gate_input))

        # Combine using gate
        output = gate * hidden + (1 - gate) * skip

        # Layer normalization
        return self.layer_norm(output)

class VariableSelectionNetwork(nn.Module):
    """
    Variable Selection Network for TFT.
    Simplified version with fewer layers.
    """
    def __init__(self, input_size_per_var, num_vars, hidden_size, output_size, dropout=0.1):
        super(VariableSelectionNetwork, self).__init__()
        self.input_size_per_var = input_size_per_var
        self.num_vars = num_vars
        self.hidden_size = hidden_size
        self.output_size = output_size

        # GRN for variable weights
        self.weight_grn = GatedResidualNetwork(
            input_size=input_size_per_var * num_vars,
            hidden_size=hidden_size,
            output_size=num_vars,
            dropout=dropout
        )

        # GRN for each variable
        self.var_grns = nn.ModuleList([
            GatedResidualNetwork(
                input_size=input_size_per_var,
                hidden_size=hidden_size,
                output_size=output_size,
                dropout=dropout
            ) for _ in range(num_vars)
        ])

    def forward(self, x):
        # x shape: [batch_size, num_vars, input_size_per_var]
        batch_size = x.size(0)
        flat_x = x.view(batch_size, -1)

        # Calculate variable weights
        var_weights = self.weight_grn(flat_x)
        var_weights = F.softmax(var_weights, dim=-1).unsqueeze(-1)  # [batch_size, num_vars, 1]

        # Transform each variable
        var_outputs = []
        for i in range(self.num_vars):
            var_outputs.append(self.var_grns[i](x[:, i]))

        var_outputs = torch.stack(var_outputs, dim=1)  # [batch_size, num_vars, output_size]

        # Weighted combination
        outputs = torch.sum(var_outputs * var_weights, dim=1)  # [batch_size, output_size]

        return outputs, var_weights

class TemporalFusionTransformer(nn.Module):
    """
    Simplified Temporal Fusion Transformer specifically for single-station forecasting.
    """
    def __init__(self, num_features, hidden_size=64, num_heads=1, 
                 dropout=0.1, forecast_horizon=24, hidden_layers=2):
        super(TemporalFusionTransformer, self).__init__()
        self.num_features = num_features
        self.hidden_size = hidden_size
        self.forecast_horizon = forecast_horizon
        self.hidden_layers = hidden_layers
        
        # Static variable processing
        self.static_var_processor = GatedResidualNetwork(
            input_size=2,  # region_code, elevation
            hidden_size=hidden_size,
            output_size=hidden_size,
            dropout=dropout
        )
        
        # Variable selection for time-varying features
        self.temporal_var_selection = VariableSelectionNetwork(
            input_size_per_var=24,  # Sequence length per feature
            num_vars=num_features,
            hidden_size=hidden_size,
            output_size=hidden_size,
            dropout=dropout
        )
        
        # LSTM encoder layers
        self.lstm_layers = nn.ModuleList([
            nn.LSTM(
                input_size=hidden_size if i == 0 else hidden_size,
                hidden_size=hidden_size,
                batch_first=True
            ) for i in range(hidden_layers)
        ])
        
        # Temporal self-attention
        self.self_attention = TemporalSelfAttention(
            d_model=hidden_size,
            n_heads=num_heads,
            dropout=dropout
        )
        
        # Final output layer for forecasting (single prediction)
        self.forecast_projection = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, forecast_horizon)
        )
        
    def forward(self, inputs):
        # Unpack inputs - simplified for single station
        temporal_features, static_features = inputs
        batch_size = temporal_features.size(0)
        
        # Process static features
        static_embeddings = self.static_var_processor(static_features)
        
        # Process temporal features with variable selection
        temporal_embeddings, temporal_weights = self.temporal_var_selection(temporal_features)
        
        # Reshape to [batch, seq_len, hidden]
        temporal_embeddings = temporal_embeddings.unsqueeze(1).expand(-1, 24, -1)
        
        # Add static embeddings to each timestep
        temporal_embeddings = temporal_embeddings + static_embeddings.unsqueeze(1)
        
        # Pass through LSTM layers
        lstm_out = temporal_embeddings
        for lstm_layer in self.lstm_layers:
            lstm_out, _ = lstm_layer(lstm_out)
        
        # Self-attention
        attention_out = self.self_attention(lstm_out)
        
        # Generate forecast
        forecast = self.forecast_projection(attention_out[:, -1, :])
        
        return forecast.unsqueeze(1)  # [batch, 1, horizon]
        
# ============================================================
# ADE OPTIMIZATION
# ============================================================
import random
from functools import partial


import torch
import numpy as np
import os
import time
from functools import partial

def objective(trial, train_loader, val_loader, feature_cols, num_stations):
    """
    Objective function for Optuna optimization.
    """
    # Sample hyperparameters
    time_steps = trial.suggest_int('time_steps', 12, 24)
    batch_size = trial.suggest_int('batch_size', 32, 64)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.05, log=True)
    hidden_size = trial.suggest_int('hidden_size', 16, 32)
    hidden_layers = trial.suggest_int('hidden_layers', 2, 3)
    
    # Create model
    model = TemporalFusionTransformer(
        num_features=len(feature_cols),
        num_stations=num_stations,
        hidden_size=hidden_size,
        num_heads=1,  # Fixed as per requirements
        dropout=0.1,
        forecast_horizon=24,
        hidden_layers=hidden_layers
    )
    
    # Train model with limited epochs for faster optimization
    model, train_losses, val_losses = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        learning_rate=learning_rate,
        epochs=10,  # Reduced from 20 to speed up optimization
        patience=3,  # Reduced from 5 to speed up optimization
        use_checkpoint=False  # Don't save checkpoints during optimization
    )
    
    # Evaluate on validation set
    model.eval()
    val_loss = 0
    val_batches = 0
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            if isinstance(inputs, tuple):
                inputs = tuple(x.to(device) for x in inputs)
            elif isinstance(inputs, list):
                inputs = [x.to(device) for x in inputs]
            else:
                inputs = inputs.to(device)

            targets = targets.to(device)
            
            outputs = model(inputs)
            
            # Calculate MAPE
            epsilon = 1e-10  # Small constant to avoid division by zero
            mape = torch.mean(torch.abs((targets - outputs) / (targets + epsilon))) * 100
            
            val_loss += mape.item()
            val_batches += 1
    
    avg_val_mape = val_loss / max(1, val_batches)
    print(f"Trial {trial.number}: Params: time_steps={time_steps}, batch_size={batch_size}, lr={learning_rate:.5f}, "
          f"hidden_size={hidden_size}, hidden_layers={hidden_layers}, Validation MAPE: {avg_val_mape:.4f}%")
    
    # Report intermediate values to enable pruning
    trial.report(avg_val_mape, step=1)
    
    # Handle pruning based on intermediate values
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
    
    return avg_val_mape

def cross_validate_model(model_class, model_kwargs, train_kwargs, feature_cols, num_stations, ts_cv, full_dataset, fold_splits):
    """
    Perform k-fold cross-validation with time series splits.
    
    Args:
        model_class: Model class to instantiate
        model_kwargs: Keyword arguments for model instantiation
        train_kwargs: Keyword arguments for training function
        feature_cols: Feature columns
        num_stations: Number of stations
        ts_cv: TimeSeriesCVSplit instance
        full_dataset: Full dataset
        fold_splits: Pre-computed fold splits
    
    Returns:
        Dictionary with cross-validation results
    """
    cv_results = []
    
    for fold_idx, (train_indices, val_indices) in enumerate(fold_splits):
        print(f"\n=== Fold {fold_idx + 1}/{len(fold_splits)} ===")
        
        # Create samplers and loaders
        train_sampler = SubsetRandomSampler(train_indices)
        val_sampler = SubsetRandomSampler(val_indices)
        
        # Create data loaders
        batch_size = train_kwargs.get('batch_size', 32)
        train_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=val_sampler)
        
        # Initialize model
        model = model_class(**model_kwargs)
        
        # Train model
        model, train_losses, val_losses = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            **{k: v for k, v in train_kwargs.items() if k != 'batch_size'}
        )
        
        # Evaluate on validation set
        model.eval()
        val_rmse, val_mae, val_r2, _ = evaluate_model(
            model, val_loader, ['LA Downtown'], {'LA Downtown': 'urban'}
        )
        
        cv_results.append({
            'fold': fold_idx + 1,
            'rmse': val_rmse,
            'mae': val_mae,
            'r2': val_r2,
            'train_size': len(train_indices),
            'val_size': len(val_indices)
        })
        
        print(f"Fold {fold_idx + 1} Results - RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}, R²: {val_r2:.4f}")
    
    # Calculate average metrics
    avg_rmse = np.mean([r['rmse'] for r in cv_results])
    avg_mae = np.mean([r['mae'] for r in cv_results])
    avg_r2 = np.mean([r['r2'] for r in cv_results])
    std_rmse = np.std([r['rmse'] for r in cv_results])
    
    print(f"\n=== Cross-Validation Results ===")
    print(f"Avg RMSE: {avg_rmse:.4f} ± {std_rmse:.4f}")
    print(f"Avg MAE: {avg_mae:.4f}")
    print(f"Avg R²: {avg_r2:.4f}")
    
    return {
        'folds': cv_results,
        'avg_rmse': avg_rmse,
        'avg_mae': avg_mae,
        'avg_r2': avg_r2,
        'std_rmse': std_rmse
    }

def optimize_hyperparameters(train_loader, val_loader, feature_cols, num_stations):
    """
    Use Optuna to optimize hyperparameters.
    """
    # Check for incompatible checkpoints
    checkpoint_path = 'checkpoint.pth'
    if os.path.exists(checkpoint_path):
        try:
            # Create a test model with the bounds midpoint
            test_model = TemporalFusionTransformer(
                num_features=len(feature_cols),
                num_stations=num_stations,
                hidden_size=24,  # Midpoint of bounds
                num_heads=1,
                dropout=0.1,
                forecast_horizon=24,
                hidden_layers=2
            )
            
            checkpoint = torch.load(checkpoint_path)
            try:
                test_model.load_state_dict(checkpoint['model_state_dict'])
                print("Checkpoint is compatible with optimization parameters.")
            except RuntimeError as e:
                if "size mismatch" in str(e):
                    print("Removing incompatible checkpoint before optimization.")
                    os.remove(checkpoint_path)
                else:
                    raise e
        except Exception as e:
            print(f"Error checking checkpoint: {e}. Removing to be safe.")
            os.remove(checkpoint_path)
    
    # Create an Optuna study with TPE sampler (more efficient than default)
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=0)
    sampler = optuna.samplers.TPESampler(seed=42)
    
    study = optuna.create_study(
        direction="minimize",
        sampler=sampler,
        pruner=pruner,
        study_name="tft_optimization"
    )
    
    # Optimize with fewer trials than ADE's population × generations
    start_time = time.time()
    study.optimize(
        func=partial(objective, train_loader=train_loader, val_loader=val_loader, 
                     feature_cols=feature_cols, num_stations=num_stations),
        n_trials=20,  # This is less than ADE's pop_size(8) × max_iter(10) = 80 evaluations
        timeout=1800,  # Add timeout of 30 minutes as safety
        n_jobs=1  # Can increase if multiple CPUs/GPUs are available
    )
    
    optimization_time = time.time() - start_time
    
    # Get best parameters
    best_params = study.best_params
    best_mape = study.best_value
    
    print("=" * 50)
    print(f"Optimization completed in {optimization_time/60:.2f} minutes")
    print("Optimal Parameters:")
    print(f"Time Steps: {best_params['time_steps']}")
    print(f"Batch Size: {best_params['batch_size']}")
    print(f"Learning Rate: {best_params['learning_rate']:.5f}")
    print(f"Hidden Size: {best_params['hidden_size']}")
    print(f"Hidden Layers: {best_params['hidden_layers']}")
    print(f"Validation MAPE: {best_mape:.4f}%")
    print("=" * 50)
    
    # Visualize the optimization process
    try:
        fig = optuna.visualization.plot_optimization_history(study)
        fig.write_image("optuna_history.png")
        
        fig = optuna.visualization.plot_param_importances(study)
        fig.write_image("optuna_param_importance.png")
        
        print("Optimization visualizations saved to files.")
    except:
        print("Visualization could not be generated. Continue without visualizations.")
    
    return {
        'time_steps': best_params['time_steps'],
        'batch_size': best_params['batch_size'],
        'learning_rate': best_params['learning_rate'],
        'hidden_size': best_params['hidden_size'],
        'hidden_layers': best_params['hidden_layers']
    }
    
def train_model(model, train_loader, val_loader, learning_rate=0.001, epochs=20, patience=5, use_checkpoint=True):
    """
    Train the model with MSE loss and early stopping.
    """
    model.to(device)
    
    checkpoint_path = 'checkpoint.pth'
    # Add a model version identifier to prevent future mismatches
    model_version = "single_output_v1"  # Change this when architecture changes
    start_epoch = 0
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Initialize optimizer and scheduler first
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=2, verbose=True
    )
     # Try to load checkpoint, but handle architecture changes gracefully
    if os.path.exists(checkpoint_path):
        try:
            checkpoint = torch.load(checkpoint_path)
            # Check if the checkpoint has a version and if it matches current model
            if 'model_version' not in checkpoint or checkpoint['model_version'] != model_version:
                print(f"Checkpoint is from a different model version. Current: {model_version}, Checkpoint: {checkpoint.get('model_version', 'unknown')}")
                print("Removing incompatible checkpoint and starting fresh training.")
                os.remove(checkpoint_path)
            else:
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                start_epoch = checkpoint['epoch'] + 1
                patience_counter = checkpoint['patience_counter']
                best_val_loss = checkpoint['best_val_loss']
                print(f"Resuming training from epoch {start_epoch}")
        except RuntimeError as e:
            if "size mismatch" in str(e):
                print("Model architecture has changed - detected parameter size mismatch.")
                print("Removing incompatible checkpoint and starting fresh training.")
                # Delete the incompatible checkpoint
                os.remove(checkpoint_path)
            else:
                # If it's some other kind of error, re-raise it
                raise e
    else:
        print("Starting training from scratch.")

   
    
    # Add mixed precision training with autocast
    scaler = torch.cuda.amp.GradScaler()

    

    train_losses = []
    val_losses = []
    epoch_times = []

    print(f"Starting training for {epochs} epochs with patience {patience}...")
    total_start_time = time.time()

    for epoch in range(start_epoch, epochs):
        epoch_start_time = time.time()
        # Training
        model.train()
        train_loss = 0
        train_batches = 0

        batch_times = []
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            batch_start = time.time()
            
            # Move to device
            if isinstance(inputs, tuple):
                inputs = tuple(x.to(device) for x in inputs)
            elif isinstance(inputs, list):
                inputs = [x.to(device) if isinstance(x, torch.Tensor) else x for x in inputs]
            else:
                inputs = inputs.to(device)

            targets = targets.to(device)

            # Forward pass with mixed precision
            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                # CHANGED: Use MSE loss instead of quantile loss
                loss = mse_loss(outputs, targets)

            # Backward pass with gradient scaling
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            train_batches += 1
            
            batch_end = time.time()
            batch_time = batch_end - batch_start
            batch_times.append(batch_time)
            
            # Print progress every 10 batches
            if (batch_idx + 1) % 10 == 0:
                print(f"  Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}, Time: {batch_time:.3f}s")

        avg_train_loss = train_loss / max(1, train_batches)
        train_losses.append(avg_train_loss)
        
        avg_batch_time = sum(batch_times) / len(batch_times) if batch_times else 0
        
        # Validation
        val_start_time = time.time()
        model.eval()
        val_loss = 0
        val_batches = 0

        with torch.no_grad():
            for inputs, targets in val_loader:
                # Move to device
                if isinstance(inputs, tuple):
                    inputs = tuple(x.to(device) for x in inputs)
                elif isinstance(inputs, list):
                    inputs = [x.to(device) if isinstance(x, torch.Tensor) else x for x in inputs]
                else:
                    inputs = inputs.to(device)
                    
                targets = targets.to(device)

                # Forward pass
                outputs = model(inputs)
                # CHANGED: Use MSE loss instead of quantile loss
                loss = mse_loss(outputs, targets)

                val_loss += loss.item()
                val_batches += 1

        avg_val_loss = val_loss / max(1, val_batches)
        val_losses.append(avg_val_loss)
        
        val_time = time.time() - val_start_time
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - epoch_start_time
        epoch_times.append(epoch_time)
        
        # Calculate estimated time remaining
        avg_epoch_time = sum(epoch_times) / len(epoch_times)
        remaining_epochs = epochs - (epoch + 1)
        est_time_remaining = avg_epoch_time * remaining_epochs
        
        # Print detailed progress
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"  Train Loss: {avg_train_loss:.4f} ({train_batches} batches, avg batch time: {avg_batch_time:.3f}s)")
        print(f"  Val Loss: {avg_val_loss:.4f} (validation time: {val_time:.2f}s)")
        print(f"  Epoch Time: {epoch_time:.2f}s, Est. Remaining: {est_time_remaining/60:.2f} minutes")
        
        # Learning rate scheduler
        scheduler.step(avg_val_loss)

        if use_checkpoint and avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Save best model with version info
            try:
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'epoch': epoch,
                    'patience_counter': patience_counter,
                    'best_val_loss': best_val_loss,
                    'model_version': model_version  # Save version with checkpoint
                }, checkpoint_path)

                print(f"  Saved best model (version {model_version}) with val loss: {best_val_loss:.4f}")
            except Exception as e:
                print(f"  Error saving model: {e}")
        else:
            patience_counter += 1
            print(f"  No improvement for {patience_counter}/{patience} epochs")
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    total_time = time.time() - total_start_time
    print(f"Training completed in {total_time/60:.2f} minutes ({total_time:.1f} seconds)")
    
        # At the end, only try to load the best model if use_checkpoint is True
    if use_checkpoint:
        try:
            if os.path.exists(checkpoint_path):
                checkpoint = torch.load(checkpoint_path)
                # Verify version before loading
                if checkpoint.get('model_version', '') == model_version:
                    model.load_state_dict(checkpoint['model_state_dict'])
                    print(f"Loaded best model (version {model_version})")
                else:
                    print("Best model checkpoint has incompatible version. Using current model state.")
        except Exception as e:
            print(f"Error loading best model: {e}")

    return model, train_losses, val_losses

def evaluate_model(model, data_loader, station_ids, regions):
    """
    Evaluate the model and calculate metrics with robust dimension handling.
    """
    model.eval()
    all_predictions = []
    all_actuals = []

    with torch.no_grad():
        for inputs, targets in data_loader:
            # Move to device - handle ALL possible input types
            if isinstance(inputs, tuple):
                inputs = tuple(x.to(device) for x in inputs)
            elif isinstance(inputs, list):
                inputs = [x.to(device) if isinstance(x, torch.Tensor) else x for x in inputs]
            else:
                inputs = inputs.to(device)

            # Forward pass with mixed precision for evaluation
            with torch.cuda.amp.autocast():
                outputs = model(inputs)

            # CHANGED: Store predictions directly instead of extracting median predictions
            preds = outputs.detach().cpu().numpy()
            targets_cpu = targets.numpy()
            
            # Append batch predictions and targets
            all_predictions.append(median_preds)
            all_actuals.append(targets_cpu)
            
            # Clear GPU cache periodically
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Process predictions in chunks to save memory
    rmse_sum = 0
    mae_sum = 0
    r2_sum = 0
    sample_count = 0
    
    # Process in smaller chunks
    for preds, acts in zip(all_predictions, all_actuals):
        # Flatten current batch
        preds_flat = preds.flatten()  
        acts_flat = acts.flatten()
        
        # Update metrics
        rmse_sum += np.sum((preds_flat - acts_flat) ** 2)
        mae_sum += np.sum(np.abs(preds_flat - acts_flat))
        sample_count += len(preds_flat)
    
    # Calculate final metrics
    rmse = np.sqrt(rmse_sum / sample_count)
    mae = mae_sum / sample_count
    
    # For R², we need all data (this is an approximation)
    all_preds_concat = np.concatenate([p.flatten() for p in all_predictions])
    all_acts_concat = np.concatenate([a.flatten() for a in all_actuals])
    r2 = r2_score(all_acts_concat, all_preds_concat)

    print(f"Overall Metrics - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

    # Since we only have one station (LA Downtown), simplify the station metrics
    station_metrics = {
        station_ids[0]: {
            'region': regions.get(station_ids[0], 'Unknown'),
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        }
    }
    
    print(f"Station {station_ids[0]} ({regions.get(station_ids[0], 'Unknown')}) - "
          f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

    return rmse, mae, r2, station_metrics

def visualize_predictions(model, data_loader, station_ids, regions, season):
    """
    Visualize predictions for each station.
    """
    model.eval()
    if len(data_loader) == 0:
        print("No data available for visualization")
        return
    
    # Get predictions
    try:
        for inputs, targets in data_loader:
            # Only process one batch for visualization
            if isinstance(inputs, tuple):
                inputs = tuple(x.to(device) for x in inputs)
            else:
                inputs = inputs.to(device)

            # Forward pass
            outputs = model(inputs)

            # Move to CPU for plotting
            outputs = outputs.cpu().numpy()
            targets = targets.numpy()
            break

        # Check if we have data to plot
        if 'outputs' not in locals():
            print("No data was loaded from the dataloader")
            return
    except Exception as e:
        print(f"Error processing visualization data: {e}")
        return
    
    # Create subplots for each station
    fig, axes = plt.subplots(len(station_ids), 1, figsize=(12, 3*len(station_ids)))
    if len(station_ids) == 1:
        axes = [axes]

    hours = np.arange(24)

    for i, station in enumerate(station_ids):
        ax = axes[i]

        # Plot actual vs predicted
        ax.plot(hours, targets[0, i, :], 'b-', label='Actual')
        ax.plot(hours, outputs[0, i, :], 'r--', label='Predicted')

        ax.set_title(f"{station} ({regions.get(station, 'Unknown')}) - {season}")
        ax.set_xlabel('Hour of Day')
        ax.set_ylabel('Temperature (°C)')
        ax.legend()
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"{season}_predictions.png")
    plt.show()  # Add this line to display the plot
    plt.close()

def create_mirror_plot(model, data_loader, station_ids, regions, season):
    """
    Create a mirror plot with actual temperature on left side and predicted on right side.
    """
    model.eval()
    if len(data_loader) == 0:
        print(f"No data available for {season} mirror plot")
        return
    
    # Get predictions
    try:
        for inputs, targets in data_loader:
            # Move inputs to device - handle ALL possible input types
            if isinstance(inputs, tuple):
                inputs = tuple(x.to(device) for x in inputs)
            elif isinstance(inputs, list):
                inputs = [x.to(device) if isinstance(x, torch.Tensor) else x for x in inputs]
            else:
                inputs = inputs.to(device)

            # Forward pass
            outputs = model(inputs)

            # Move to CPU for plotting
            outputs = outputs.cpu().numpy()
            targets = targets.numpy()
            break
            
        if 'outputs' not in locals():
            print(f"No data was loaded for {season} mirror plot")
            return
    except Exception as e:
        print(f"Error processing mirror plot data for {season}: {e}")
        import traceback
        traceback.print_exc()  # Print full error details
        return
    
    # Create a figure for each station
    for i, station in enumerate(station_ids):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
        
        hours = np.arange(24)
        
        # Left plot - Actual temperatures
        ax1.plot(hours, targets[0, i, :], 'b-o', linewidth=2)
        ax1.set_title(f"Actual Temperature - {season}", fontsize=14)
        ax1.set_xlabel('Hour of Day', fontsize=12)
        ax1.set_ylabel('Temperature (°C)', fontsize=12)
        ax1.set_xlim(0, 23)
        ax1.grid(True, alpha=0.3)
        ax1.invert_xaxis()  # Invert x-axis for mirror effect
        
        # Right plot - Predicted temperatures
        ax2.plot(hours, outputs[0, i, :], 'r-o', linewidth=2)
        ax2.set_title(f"Predicted Temperature - {season}", fontsize=14)
        ax2.set_xlabel('Hour of Day', fontsize=12)
        ax2.set_xlim(0, 23)
        ax2.grid(True, alpha=0.3)
        
        # Add overall title
        plt.suptitle(f"{station} ({regions.get(station, 'Unknown')}) - {season} Day Comparison", 
                    fontsize=16, y=1.05)
        
        # Add a line in the middle
        fig.tight_layout()
        plt.subplots_adjust(wspace=0.05)  # Reduce space between subplots
        
        # Save and display
        plt.savefig(f"{season}_mirror_comparison.png", bbox_inches='tight')
        plt.show()
        plt.close()

def create_seasonal_datasets(df, target_days, all_stations, feature_cols):
    """
    Create datasets for each seasonal day and visualize predictions.
    """
    seasonal_datasets = {}
    
    for season, day_data in target_days.items():
        if day_data.empty:
            print(f"No data available for {season}")
            continue
            
        # Get the date of this target day
        sample_date = day_data['timestamp'].iloc[0]
        print(f"Creating dataset for {season}: {sample_date.strftime('%Y-%m-%d')}")
        
        try:
            # Prepare the data in the format expected by the model
            # Format should be: temporal_features [batch, features, stations, time], static_features [batch, stations, static_features]
            
            # Reshape features to match expected format
            temporal_data = np.zeros((1, len(feature_cols), 1, 24))  # [batch=1, features, stations=1, time=24]
            
            # Fill in the temporal features
            for f_idx, feat in enumerate(feature_cols):
                if feat in day_data.columns:
                    temporal_data[0, f_idx, 0, :] = day_data[feat].values[:24]  # Using first 24 records
            
            # Create static features (region_code and elevation)
            static_data = np.zeros((1, 1, 2))  # [batch=1, stations=1, static_features=2]
            static_data[0, 0, 0] = day_data['region_code'].iloc[0] 
            static_data[0, 0, 1] = day_data['elevation_norm'].iloc[0]
            
            # Create target (actual temperatures)
            target_data = np.zeros((1, 1, 24))  # [batch=1, stations=1, time=24]
            target_data[0, 0, :] = day_data['Temperature_C'].values[:24]  # Using first 24 records
            
            # Convert to tensors
            X_temporal = torch.FloatTensor(temporal_data)
            X_static = torch.FloatTensor(static_data)
            y = torch.FloatTensor(target_data)
            
            # Store as a list containing a single data point
            seasonal_datasets[season] = [((X_temporal, X_static), y)]
            print(f"Created {season} dataset with shapes: X_temporal={X_temporal.shape}, X_static={X_static.shape}, y={y.shape}")
            
        except Exception as e:
            print(f"Error creating {season} dataset: {e}")
            import traceback
            traceback.print_exc()
    
    return seasonal_datasets


def analyze_topographic_performance(station_metrics, regions):
    """
    Analyze model performance across different topographic regions.
    """
    # Group metrics by region
    region_metrics = {}
    for station, metrics in station_metrics.items():
        region = regions.get(station, 'Unknown')
        if region not in region_metrics:
            region_metrics[region] = []
        region_metrics[region].append(metrics)

    # Calculate average metrics by region
    region_avg_metrics = {}
    for region, metrics_list in region_metrics.items():
        avg_rmse = np.mean([m['rmse'] for m in metrics_list])
        avg_mae = np.mean([m['mae'] for m in metrics_list])
        avg_r2 = np.mean([m['r2'] for m in metrics_list])

        region_avg_metrics[region] = {
            'avg_rmse': avg_rmse,
            'avg_mae': avg_mae,
            'avg_r2': avg_r2
        }

        print(f"Region {region} - Avg RMSE: {avg_rmse:.4f}, Avg MAE: {avg_mae:.4f}, Avg R²: {avg_r2:.4f}")

        if not station_metrics:
            print("No station metrics available for analysis")
            return {}

    # Create bar chart comparing regions
    regions = list(region_avg_metrics.keys())
    rmse_values = [region_avg_metrics[r]['avg_rmse'] for r in regions]

    plt.figure(figsize=(10, 6))
    bars = plt.bar(regions, rmse_values)

    # Add styling
    plt.title('RMSE by Topographic Region', fontsize=16)

    plt.title('RMSE by Topographic Region', fontsize=16)
    plt.ylabel('RMSE (°C)', fontsize=14)
    plt.xlabel('Region', fontsize=14)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height:.2f}',
                 ha='center', va='bottom', fontsize=12)

    plt.tight_layout()
    plt.show()
    plt.savefig('region_performance.png')
    plt.close()

    return region_avg_metrics

def analyze_seasonal_performance(seasonal_results):
    """
    Compare model performance across different seasons.
    """
    seasons = list(seasonal_results.keys())
    rmse_values = [results['rmse'] for results in seasonal_results.values()]
    mae_values = [results['mae'] for results in seasonal_results.values()]

    if not seasonal_results:
        print("No seasonal results available for analysis")
        return [], [], []

    # Create grouped bar chart
    x = np.arange(len(seasons))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(x - width/2, rmse_values, width, label='RMSE')
    ax.bar(x + width/2, mae_values, width, label='MAE')

    ax.set_title('Model Performance by Season', fontsize=16)
    ax.set_ylabel('Error (°C)', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(seasons)
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Add value labels
    for i, v in enumerate(rmse_values):
        ax.text(i - width/2, v + 0.1, f'{v:.2f}', ha='center', fontsize=10)

    for i, v in enumerate(mae_values):
        ax.text(i + width/2, v + 0.1, f'{v:.2f}', ha='center', fontsize=10)

    plt.tight_layout()
    plt.show()
    plt.savefig('seasonal_performance.png')
    plt.close()

    return seasons, rmse_values, mae_values




c:\Users\jamak\Downloads\Ana\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Using device: cpu


In [4]:
# ============================================================
# BLOCK 1: INITIALIZATION AND DATA LOADING
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
from torch.utils.data import DataLoader, random_split

print("LA Downtown Weather Forecasting with VMD + TFT")
print("=" * 70)

# Load and prepare data
df = load_data(DATA_PATH)
print(f"Loaded data with {len(df)} records")

# All stations - only LA Downtown in this case
all_stations = ['LA Downtown']
regions = {'LA Downtown': 'urban'}

# Define feature columns to use
feature_cols = [
    'Temperature_C',
    'HourlyRelativeHumidity',
    'HourlyStationPressure',
    'hour_sin', 'hour_cos',
    'day_sin', 'day_cos'
]

print("BLOCK 1 COMPLETED: Initialization and data loading successful.")

LA Downtown Weather Forecasting with VMD + TFT
Filtered data to LA Downtown only
Loaded data with 35040 records
BLOCK 1 COMPLETED: Initialization and data loading successful.


In [5]:
# ============================================================
# BLOCK 2: VMD DECOMPOSITION
# ============================================================

# Apply VMD decomposition to the Temperature_C feature
print("Performing VMD decomposition...")
decomposed_data = parallel_vmd_decomposition(df, feature_cols_to_decompose=['Temperature_C'])

# Expand feature columns with decomposed modes
expanded_feature_cols = feature_cols.copy()

# For LA Downtown, add decomposed temperature modes to feature columns
la_downtown_modes = decomposed_data['LA Downtown']['Temperature_C']
n_modes = la_downtown_modes.shape[0]  # Number of VMD modes

print(f"Found {n_modes} VMD modes for Temperature_C")

# Create expanded dataset with VMD modes
vmd_df = df.copy()

# Add VMD modes as new features
for i in range(n_modes):
    mode_name = f'Temp_Mode_{i+1}'
    vmd_df[mode_name] = la_downtown_modes[i, :]
    expanded_feature_cols.append(mode_name)

print("BLOCK 2 COMPLETED: VMD decomposition successful.")

Performing VMD decomposition...
Performing VMD decomposition for feature: Temperature_C
  Processing station: LA Downtown
Starting decomposition for LA Downtown - Temperature_C
Loading cached VMD decomposition for LA Downtown - Temperature_C
Found 14 VMD modes for Temperature_C
BLOCK 2 COMPLETED: VMD decomposition successful.


In [6]:
# ============================================================
# BLOCK 3: DATASET CREATION AND SPLITTING
# ============================================================
from torch.utils.data import DataLoader, SubsetRandomSampler

# Create dataset with expanded features
full_dataset = WeatherDataset(
    df=vmd_df,
    station_ids=all_stations,
    feature_cols=expanded_feature_cols,
    seq_length=24,
    forecast_horizon=24
)

# First, create data loaders with default batch size
initial_batch_size = 32  # Default value before optimization

# Create time series CV splits
ts_cv = TimeSeriesCVSplit(
    timestamps=full_dataset.timestamps, 
    n_splits=5,
    test_size=0.15, 
    gap=24,  # 24-hour gap between train and validation
    min_train_size=0.6
)

# Visualize the CV folds
fold_info = ts_cv.get_fold_timestamps()
for fold in fold_info:
    print(f"Fold {fold['fold']}:")
    print(f"  Train: {fold['train_range'][0]} to {fold['train_range'][1]}")
    print(f"  Validation: {fold['val_range'][0]} to {fold['val_range'][1]}")

# Create samplers and loaders for the first fold (for initial training)
fold_splits = list(ts_cv.split())
train_indices, val_indices = fold_splits[0]

# Reserve the last fold for final testing
test_indices = fold_splits[-1][1]

# Create data loaders with the samplers
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Create data loaders with the samplers
train_dataset = DataLoader(full_dataset, batch_size=initial_batch_size, sampler=train_sampler)
val_dataset = DataLoader(full_dataset, batch_size=initial_batch_size, sampler=val_sampler)
test_dataset = DataLoader(full_dataset, batch_size=initial_batch_size, sampler=test_sampler)

print(f"Time-based CV split created: {len(train_indices)} training samples, "
      f"{len(val_indices)} validation samples, {len(test_indices)} test samples")

train_loader = DataLoader(
    train_dataset, 
    batch_size=initial_batch_size, 
    shuffle=True,
    pin_memory=True,
    num_workers=2,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=initial_batch_size, 
    shuffle=False,
    pin_memory=True,
    num_workers=2,
    persistent_workers=True
)

print("BLOCK 3 COMPLETED: Dataset creation and splitting successful.")


Created dataset with 34946 valid windows
Fold 1:
  Train: 2021-01-02 00:00:00 to 2023-05-26 23:00:00
  Validation: 2023-05-28 00:00:00 to 2024-01-02 00:00:00
Fold 2:
  Train: 2021-01-02 00:00:00 to 2023-08-26 06:00:00
  Validation: 2023-08-27 07:00:00 to 2024-04-02 06:00:00
Fold 3:
  Train: 2021-01-02 00:00:00 to 2023-11-25 12:00:00
  Validation: 2023-11-26 13:00:00 to 2024-07-02 12:00:00
Fold 4:
  Train: 2021-01-02 00:00:00 to 2024-02-24 18:00:00
  Validation: 2024-02-25 19:00:00 to 2024-10-01 18:00:00
Fold 5:
  Train: 2021-01-02 00:00:00 to 2024-05-26 00:00:00
  Validation: 2024-05-27 01:00:00 to 2025-01-01 00:00:00
Time-based CV split created: 21000 training samples, 5256 validation samples, 5256 test samples
BLOCK 3 COMPLETED: Dataset creation and splitting successful.


In [ ]:
# Define the quick test function
def quick_test_run(train_loader, val_loader, feature_cols, num_stations):
    """Run a quick test to ensure training works."""
    print("Running quick test to verify checkpoint compatibility...")
    
    # Create a small model with fixed parameters
    test_model = TemporalFusionTransformer(
        num_features=len(feature_cols),
        hidden_size=24,
        num_heads=1,
        dropout=0.1,
        forecast_horizon=24,
        hidden_layers=2
    )
    
    # Train for only 2 epochs with early stopping disabled
    model, train_losses, val_losses = train_model(
        model=test_model,
        train_loader=train_loader,
        val_loader=val_loader,
        learning_rate=0.01,
        epochs=2,  # Just 2 epochs for quick testing
        patience=5,  # High patience to avoid early stopping
        use_checkpoint=False  # Set to False to avoid checkpoint issues
    )
    
    print("Test run completed successfully!")
    return True

# First run the quick test to verify everything works
print("BLOCK 3.5: Running quick test before optimization...")
quick_test_run(train_loader, val_loader, expanded_feature_cols, len(all_stations))

BLOCK 3.5: Running quick test before optimization...
Running quick test to verify checkpoint compatibility...
Starting training from scratch.
Starting training for 2 epochs with patience 5...


In [ ]:
# ============================================================
# BLOCK 4: HYPERPARAMETER OPTIMIZATION
# ============================================================

# Run ADE optimization to find best hyperparameters
print("Running ADE optimization for hyperparameters...")
best_params = optimize_hyperparameters(
    train_loader=train_loader,
    val_loader=val_loader,
    feature_cols=expanded_feature_cols
)

print("Optimized hyperparameters:")
for param, value in best_params.items():
    print(f"- {param}: {value}")
    
print("BLOCK 4 COMPLETED: Hyperparameter optimization successful.")

Running ADE optimization for hyperparameters...
Removing incompatible checkpoint before optimization.
Starting training from scratch.
Starting training for 5 epochs with patience 2...
  Batch 10/765, Loss: 60.4488, Time: 0.069s
  Batch 20/765, Loss: 29.6770, Time: 0.084s
  Batch 30/765, Loss: 18.9761, Time: 0.071s
  Batch 40/765, Loss: 18.9913, Time: 0.086s
  Batch 50/765, Loss: 19.8823, Time: 0.072s
  Batch 60/765, Loss: 21.7851, Time: 0.078s
  Batch 70/765, Loss: 22.8416, Time: 0.071s
  Batch 80/765, Loss: 19.0067, Time: 0.072s
  Batch 90/765, Loss: 30.1333, Time: 0.054s
  Batch 100/765, Loss: 25.2879, Time: 0.068s
  Batch 110/765, Loss: 31.4014, Time: 0.068s
  Batch 120/765, Loss: 28.7064, Time: 0.072s
  Batch 130/765, Loss: 28.7608, Time: 0.069s
  Batch 140/765, Loss: 30.9138, Time: 0.044s
  Batch 150/765, Loss: 25.6082, Time: 0.068s
  Batch 160/765, Loss: 25.1586, Time: 0.071s
  Batch 170/765, Loss: 25.9122, Time: 0.071s
  Batch 180/765, Loss: 24.8534, Time: 0.071s
  Batch 190/765

In [ ]:
# BLOCK 5: MODEL TRAINING WITH OPTIMIZED PARAMETERS
# ============================================================

# Now recreate data loaders with optimized batch size
train_loader = DataLoader(
    train_dataset, 
    batch_size=best_params['batch_size'], 
    shuffle=True,
    pin_memory=True,
    num_workers=2,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=best_params['batch_size'], 
    shuffle=False,
    pin_memory=True,
    num_workers=2,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=1,  # Usually keep test batch size at 1 for detailed evaluation
    shuffle=False,
    pin_memory=True
)

# Create model with optimized parameters
model = TemporalFusionTransformer(
    num_features=len(expanded_feature_cols),
    hidden_size=best_params['hidden_size'],
    num_heads=1,  # Fixed as per requirements
    dropout=0.1,
    forecast_horizon=24,
    hidden_layers=best_params['hidden_layers'] 
)

# Train model with optimized parameters
print(f"Training optimized TFT model...")
model, train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    learning_rate=best_params['learning_rate'],
    epochs=20,
    patience=5
)

print("BLOCK 5 COMPLETED: Model training successful.")

In [ ]:
# ============================================================
# BLOCK 6: LOSS VISUALIZATION
# ============================================================

# Plot training curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss - LA Downtown')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('training_validation_loss.png')
plt.show()

print("BLOCK 6 COMPLETED: Loss visualization successful.")


In [ ]:
# ============================================================
# BLOCK 7: MODEL EVALUATION
# ============================================================

# Evaluate model
print(f"Evaluating model on LA Downtown...")
rmse, mae, r2, station_metrics = evaluate_model(
    model=model,
    data_loader=test_loader,
    station_ids=all_stations,
    regions=regions
)

print(f"Overall metrics - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

# Visualize predictions
visualize_predictions(
    model=model,
    data_loader=test_loader,
    regions=regions,
    season="All Seasons - LA Downtown"
)

print("BLOCK 7 COMPLETED: Model evaluation successful.")

In [ ]:
# ============================================================
# BLOCK 8: SEASONAL ANALYSIS - DATASET CREATION
# ============================================================

print("\nCreating seasonal datasets...")
seasonal_datasets = create_seasonal_datasets(df, target_days, all_stations, feature_cols)

# Visualize predictions for each season
for season, season_loader in seasonal_datasets.items():
    print(f"Visualizing predictions for {season}...")
    visualize_predictions(
        model=model,
        data_loader=season_loader,
        station_ids=all_stations,
        regions=regions,
        season=season
    )

print("BLOCK 8 COMPLETED: Seasonal datasets creation successful.")


In [ ]:
# ============================================================
# BLOCK 9: SEASONAL PERFORMANCE EVALUATION
# ============================================================

# Evaluate for each season
seasonal_results = {}
for season, season_loader in seasonal_datasets.items():
    if len(season_loader) > 0:
        print(f"Evaluating model on {season} data...")
        rmse, mae, r2, _ = evaluate_model(
            model=model,
            data_loader=season_loader,
            station_ids=all_stations,
            regions=regions
        )
        seasonal_results[season] = {
            'rmse': rmse,
            'mae': mae,
            'r2': r2
        }
        print(f"{season} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

print("BLOCK 9 COMPLETED: Seasonal performance evaluation successful.")

In [ ]:
# ============================================================
# BLOCK 10: SEASONAL MIRROR PLOTS
# ============================================================

# Function to unnormalize temperature data
def unnormalize_temperature(temp_normalized, temp_min, temp_max):
    """
    Convert normalized temperature back to original scale.
    """
    return temp_normalized * (temp_max - temp_min) + temp_min

# Get the temperature normalization parameters
temp_min = df['Temperature_C'].min()
temp_max = df['Temperature_C'].max()
print(f"Temperature normalization range: min={temp_min:.2f}, max={temp_max:.2f}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for season, seasonal_data in seasonal_datasets.items():
    print(f"Processing {season} forecast...")
    
    try:
        # This is a list with a single data point
        (X_temporal, X_static), y = seasonal_data[0]
        
        # Move to device
        X_temporal = X_temporal.to(device)
        X_static = X_static.to(device)
        
        # Forward pass
        model.eval()
        with torch.no_grad():
            outputs = model((X_temporal, X_static))
        
        # Move to CPU for plotting
        outputs = outputs.cpu().numpy()
        y_np = y.numpy()
        
        # Unnormalize the temperature data
        outputs_unnorm = unnormalize_temperature(outputs, temp_min, temp_max)
        y_np_unnorm = unnormalize_temperature(y_np, temp_min, temp_max)
        
        # Create the mirror plot with unnormalized data
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
        
        hours = np.arange(24)
        
        # Left plot - Actual temperatures
        ax1.plot(hours, y_np_unnorm[0, 0, :], 'b-o', linewidth=2)
        ax1.set_title(f"Actual Temperature - {season}", fontsize=14)
        ax1.set_xlabel('Hour of Day', fontsize=12)
        ax1.set_ylabel('Temperature (°C)', fontsize=12)
        ax1.set_xlim(0, 23)
        ax1.grid(True, alpha=0.3)
        ax1.invert_xaxis()  # Invert x-axis for mirror effect
        
        # Right plot - Predicted temperatures
        ax2.plot(hours, outputs_unnorm[0, 0, :], 'r-o', linewidth=2)
        ax2.set_title(f"Predicted Temperature - {season}", fontsize=14)
        ax2.set_xlabel('Hour of Day', fontsize=12)
        ax2.set_xlim(0, 23)
        ax2.grid(True, alpha=0.3)
        
        # Add overall title
        plt.suptitle(f"{all_stations[0]} ({regions.get(all_stations[0], 'Unknown')}) - {season} Day Comparison", 
                    fontsize=16, y=1.05)
        
        # Tight layout with minimal space between plots
        fig.tight_layout()
        plt.subplots_adjust(wspace=0.05)  # Reduce space between subplots
        
        # Add a legend to explain colors
        ax1.plot([], [], 'b-o', label='Actual Temperature')
        ax2.plot([], [], 'r-o', label='Predicted Temperature')
        ax1.legend(loc='upper left')
        ax2.legend(loc='upper right')
        
        # Save and display
        plt.savefig(f"{season}_mirror_comparison.png", bbox_inches='tight')
        plt.show()
        plt.close()
        
    except Exception as e:
        print(f"Error processing {season} forecast: {e}")
        import traceback
        traceback.print_exc()

print("BLOCK 10 COMPLETED: Seasonal mirror plots created successfully.")

In [ ]:
# ============================================================
# BLOCK 11: FINAL ANALYSIS AND RESULTS SAVING
# ============================================================

# Analyze seasonal performance
if seasonal_results:
    analyze_seasonal_performance(seasonal_results)

print("Skipping topographic analysis since we only have data for LA Downtown")

# Save results to CSV
results_df = pd.DataFrame([{
    'Station': 'LA Downtown',
    'Region': 'urban',
    'RMSE': rmse,
    'MAE': mae,
    'R2': r2
}])

results_df.to_csv('la_downtown_results.csv', index=False)
print("Results saved to la_downtown_results.csv")

print("BLOCK 11 COMPLETED: Final analysis and results saving successful.")
print("=" * 70)
print("ALL BLOCKS EXECUTED SUCCESSFULLY! Analysis complete.")